<a href="https://colab.research.google.com/github/tsa87/anb-net/blob/master/google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jan 19 04:01:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install torch==1.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install dive-into-graphs

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.4 MB/s eta 0:11:42tcmalloc: large alloc 1147494400 bytes == 0x55d89b5ae000 @  0x7f89fc107615 0x55d8624174cc 0x55d8624f747a 0x55d86241a2ed 0x55d86250be1d 0x55d86248de99 0x55d8624889ee 0x55d86241bbda 0x55d86248dd00 0x55d8624889ee 0x55d86241bbda 0x55d86248a737 0x55d86250cc66 0x55d862489daf 0x55d86250cc66 0x55d862489daf 0x55d86250cc66 0x55d862489daf 0x55d86241c039 0x55d86245f409 0x55d86241ac52 0x55d86248dc25 0x55d8624889ee 0x55d86241bbda 0x55d86248a737 0x55d8624889ee 0x55d86241bbda 0x55d862489915 0x55d86241bafa 0x55d862489c0d 0x55d8624889ee
     |██████████████████▌             | 1055.7 MB 1.4 MB/s eta 0:09:27tcmalloc: large alloc 1434370048 bytes == 0x55d8dfc04000 @  0x7f89fc107615 0x55d8624174cc 0x55d8624f747a 0x55d86241a2ed 0x55d86250be1d 0x55d86248de99 0x55d8624889ee 0x55d86241bbda 0x55d86248dd00 0x55d8624889ee 0x55d86241bbda 0x55d86248a737 0x55d86250cc66 0x55d8

In [4]:
!git clone https://github.com/tsa87/anb-net
%cd anb-net

Cloning into 'anb-net'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 101 (delta 31), reused 97 (delta 27), pack-reused 0
Receiving objects: 100% (101/101), 7.89 MiB | 24.42 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/anb-net


In [5]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse

import numpy as np
import torch

from dig.ggraph.dataset import ZINC250k, ZINC800
from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

_ = ZINC800(
        one_shot=False,
        use_aug=False,
        root="../data/zinc",
    )

zinc_250_jt = torch.load(
    os.path.join("../data/zinc", "zinc_800_jt/processed/data.pt")
)
smiles = zinc_250_jt[-1]
labels = zinc_250_jt[0].y

making raw files: ../data/zinc/raw


Processing...
making processed files: ../data/zinc/zinc_800_jt/processed
Done!


In [6]:
jtvae = SemiJTVAEGeneratorPredictor(smiles)

In [ ]:
preprocessed, labels = jtvae.preprocess(smiles, labels)

In [ ]:
jtvae.get_model(
    "rand_gen",
    {
        "hidden_size": conf["hidden_size"],
        "latent_size": conf["latent_size"],
        "depthT": conf["depthT"],
        "depthG": conf["depthG"],
        "label_size": 1,
        "label_mean": 0,
        "label_var": 1,
    },
)

In [ ]:
loader = SemiMolTreeFolder(
    preprocessed,
    labels,
    jtvae.vocab,
    conf["batch_size"],
    num_workers=conf["num_workers"],
)

In [ ]:
print("Training model...")
jtvae.train_gen_pred(
    loader=loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate = 0.9,
    clip_norm = 50.0,
    num_epochs = 5,
    alpha = 0.1,
    beta = 0.0,
    max_beta = 1.0,
    step_beta = 0.002,
    anneal_iter = 40000,
    kl_anneal_iter = 2000,
    print_iter = 1,
    save_iter = 5000
)